In [143]:
%pip install tensorboardX --user
%pip install bs4 --user
%pip install plotly --user
%pip install -U auquan_toolbox --user


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [144]:
%pip install auquan-toolbox --user
%pip show auquan-toolbox

Note: you may need to restart the kernel to use updated packages.


Name: auquan-toolbox
Version: 2.1.92
Summary: Auquan Toolbox for developing strategies and backtesting
Home-page: https://auquan.com
Author: Auquan Team
Author-email: info@auquan.com
License: MIT
Location: c:\users\hp\appdata\roaming\python\python38\site-packages
Requires: lxml, tensorboard, numpy, scipy, sklearn, requests, pandas, plotly, tensorboardX, bs4
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [145]:
import sys
sys.path.append(r'c:\users\hp\appdata\roaming\python\python38\site-packages')
print(sys.path)

['c:\\Users\\hp\\Documents\\GitHub\\trading x ml', 'd:\\PROGRAMMING\\anaconda\\python38.zip', 'd:\\PROGRAMMING\\anaconda\\DLLs', 'd:\\PROGRAMMING\\anaconda\\lib', 'd:\\PROGRAMMING\\anaconda', '', 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python38\\site-packages', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\win32', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\win32\\lib', 'd:\\PROGRAMMING\\anaconda\\lib\\site-packages\\Pythonwin', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages', 'c:\\users\\hp\\appdata\\roaming\\python\\python38\\site-packages']


In [146]:
from backtester.trading_system_parameters import TradingSystemParameters
from backtester.features.feature import Feature
from backtester.timeRule.custom_time_rule import CustomTimeRule
from backtester.executionSystem.simple_execution_system import SimpleExecutionSystem
from backtester.orderPlacer.backtesting_order_placer import  BacktestingOrderPlacer
from backtester.trading_system import TradingSystem
from backtester.dataSource.yahoo_data_source import YahooStockDataSource
from backtester.constants import *




In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

In [153]:
class MyTradingParams(TradingSystemParameters):

    def __init__(self,tradefns):
        self.__tradefns=tradefns
        super(MyTradingParams, self).__init__()
        #self.__dataSetID = 'equity_data'
        self.__instrumentIds = self.__tradefns.getSymbolsToTrade()
       # self.__startDate = '2015/01/02'
       # self.__endDate = '2017/08/31'

    """def getDataParser(self):
        instrumentIds = self.__tradefns.getSymbolsToTrade()
        return YahooStockDataSource( 
            cachedFolderName = 'yahooData',
            dataSetId = self.__dataSetID,
            instrumentIds = instrumentIds,
            startDateStr = self.__startDate,
            endDateStr = self.__endDate,)"""
    
    def getDataParser(self):
        instrumentIds = ['AMZN']
        startDateStr= '2021/01/02'
        endDateStr= '2022/08/31'
        instrumentIds=list(set(instrumentIds+[self.getBenchmark()]))
        return YahooStockDataSource(cachedFolderName='yahooData',
                                    dataSetId = 'AquanToolboxTest',               
                                    instrumentIds=instrumentIds,
                                    startDateStr=startDateStr,
                                    endDateStr=endDateStr)
    
    def getBenchmark(self):
        return 'SPY'
    
    def getTimeRuleForUpdates(self):
        return CustomTimeRule(
            startDate =self.__startDate,
            endDate = self.__endDate,
            frequency = 'D',
            sample= '30'
        )
    
    def getFrequencyOffeatureUpdates():
        return timedelta( days =1)
    
    def getStartingCapital(self):
        if len(self.__tradefns.getSymbolsToTrade()) > 0:
            return 1000*len(self.__tradefns.getSymbolsToTrade())
        else:
            return 30000
        
    def getCustomFeatures(self):
        return  {
            'my_custom_feature': MyCustomFeature,
            'prediction': TrainingPredictionFeature,
            'zero_fees': FeesCalculator,
            'benchmark_PnL': BuyHoldPnL,
            'score': ScoreFeature
        }
    def getInstrumentFeatureConfigDicts(self):
        
        predictionDict = {
            'featureKey': 'prediction',
            'featureId': 'prediction',
            'params': {}
        }
        feesConfigDict = {
            'featureKey': 'fees',
            'featureId': 'zero_fees',
            'params': {}
        }
        profitlossConfigDict = {
            'featureKey': 'pnl',
            'featureId': 'pnl',
            'params': {
                'price': self.getPriceFeatureKey(),
                'fees': 'fees'
            }
        }
        capitalConfigDict = {
            'featureKey': 'capital',
            'featureId': 'capital',
            'params': {
                'price': 'adjClose',
                'fees': 'fees',
                'capitalReqPercent': 0.95
            }
        }
        benchmarkDict = {
            'featureKey': 'benchmark',
            'featureId': 'benchmark_PnL',
            'params': {'pnlKey': 'pnl'}
        }
        scoreDict = {
            'featureKey': 'score',
            'featureId': 'score',
            'params': {
                'featureName1': 'pnl',
                'featureName2':'benchmark'
            }
        }

        stockFeatureConfigs = self.__tradefns.getInstrumentFeatureConfigDicts()

        return {
            INSTRUMENT_TYPE_STOCK: stockFeatureConfigs + [
                predictionDict,
                feesConfigDict,
                profitlossConfigDict,
                capitalConfigDict,
                benchmarkDict, 
                scoreDict
            ]
        }
    
    def getMarketFeatureConfigDicts(self):
          scoreDict = {
            'featureKey': 'score',
            'featureId': 'score_ll',
            'params': {
                'featureName': self.getPriceFeatureKey(),
                'instrument_score_feature': 'score'
            }
        }
          return [scoreDict]

    def getPrediction(self, time, updateNum, instrumentManager):
        predictions = pd.Series(index = self.__instrumentIds)
        predictions = self.__tradefns.getPrediction(time, updateNum, instrumentManager, predictions)
        return predictions
    
    def getExecutionSystem(self):
            return SimpleExecutionSystem(
            enter_threshold = 0.7,
            exit_threshold = 0.55,
            longLimit = 1,
            shortLimit = 1,
            capitalUsageLimit = 0.10*self.getStartingCapital(),
            enterlotSize = 1, 
            exitlotSize = 1,
            limitType = 'L', 
            price = 'adjClose'
        )
    def getOrderPlacer(self):
        return BacktestingOrderPlacer
    
    def getLookbackSize(self):
        return 120
    
    def getPriceFeatureKey(self):
        return 'adjClose'
    
    def getInstrumentsIds(self):
        
        return self.__instrumentIds



In [154]:
class TrainingPredictionFeature(Feature):
    @classmethod
    def computeInstruments(cls,updateNum,time,featureParams,featureKey,instrumentManager):
        tf= MyFunctions()
        tp= MyTradingParams(tf)

class FeesCalculator(Feature):
    @classmethod
    def computeforinstruments(cls,updateNum,time,featureParams,featureKey,instrumentManager):
        return pd.Series(0,index=instrumentManager.getAllInstrumentId())
    
class BuyHoldPnL(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()

        priceData = instrumentLookbackData.getFeatureDf('adjClose')
         
        if len(priceData) < 2:
            return pd.Series(0, index = instrumentManager.getAllInstrumentsByInstrumentId())
        else:
            bhpnl = instrumentLookbackData.getFeatureDf(featureKey).iloc[-1]
            bhpnl += priceData.iloc[-1] - priceData.iloc[-2]

        return bhpnl
    
    
class ScoreFeature(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()
        if len(instrumentLookbackData.getFeatureDf(featureParams['featureName1'])) > 0:
            feature1 = instrumentLookbackData.getFeatureDf(featureParams['featureName1']).iloc[-1]
            feature2 = instrumentLookbackData.getFeatureDf(featureParams['featureName2']).iloc[-1]
            
            for instrumentId in feature1.index:
                pnls = instrumentLookbackData.getFeatureDf('pnl')[instrumentId]
                positions = instrumentLookbackData.getFeatureDf('position')[instrumentId]
                
                print(instrumentId)
                print('pnl: %.2f'%pnls[-1])
                if len(positions) > 2 and np.abs(positions[-1] - positions[-2]) > 0:
                    print('Position changed to: %.2f'%positions[-1])
            
            toRtn = (feature1 - feature2) / feature2.abs()
            toRtn[toRtn.isnull()] = 0
            toRtn[toRtn == np.Inf] = 0
        else:
            toRtn=0

        return toRtn


In [155]:
class MyFunctions():
    def __init__(self):
        self.count = 0
        self.params = {}

    def getSymbolsToTrade(self):
        return['AMZN']
        

    def getInstrumentFeatureConfigDicts(self):
        ma1Dict = {
            'featureKey': 'ma_90',
            'featureId': 'moving_average',
            'params': {
                'period': 90,
                'featureName': 'adjClose'
            }
        }
        mom30Dict = {
            'featureKey': 'mom_30',
            'featureId': 'momentum',
            'params': {
                'period': 30,
                'featureName': 'adjClose'
            }
        }
        mom10Dict = {
            'featureKey': 'mom_10',
            'featureId': 'momentum',
            'params': {
                'period': 10,
                'featureName': 'adjClose'
            }
        }
        rsiDict = {
            'featureKey': 'rsi_10',
            'featureId': 'rsi',
            'params': {
                'period': 30,
                'featureName': 'adjClose'
            }
        }
        return [ma1Dict, mom10Dict, mom30Dict,rsiDict]
    
    def getPredictions(self, time, updateNum, instrumentManager, predictions):
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
 
        def hurst_exp(input_ts, lags_to_test=20):
            tau = []
            lagvec = []   
            for lag in range(2, lags_to_test):  
                pp = np.subtract(input_ts[lag:], input_ts[:-lag])  
                lagvec.append(lag)  
                tau.append(np.sqrt(np.std(pp)))  
            m = np.polyfit(np.log10(lagvec), np.log10(tau), 1)  
            hurst = m[0]*2
            return hurst
    
        mom30Data = lookbackInstrumentFeatures.getFeatureDf('mom_30')
        ma90Data = lookbackInstrumentFeatures.getFeatureDf('ma_90')


        if len(ma90Data.index) > 20:
            mom30 = mom30Data.iloc[-1]
            # Go long if momentum is positive
            predictions[mom30 > 0] = 1
            # Go short if momentum is negative
            predictions[mom30 <= 0] = 0          
        else:
            # If no sufficient data then don't take any positions
            predictions.values[:] = 0.5
        return predictions
    
    def updateCount(self):
        self.count = self.count + 1



        
        


In [156]:
class MyCustomFeature(Feature):
    ''''
    Custom Feature to implement for instrument. This function would return the value of the feature you want to implement.
    1. create a new class MyCustomFeatureClassName for the feature and implement your logic in the function computeForInstrument() -

    2. modify function getCustomFeatures() to return a dictionary with Id for this class
        (follow formats like {'my_custom_feature_identifier': MyCustomFeatureClassName}.
        Make sure 'my_custom_feature_identifier' doesnt conflict with any of the pre defined feature Ids

        def getCustomFeatures(self):
            return {'my_custom_feature_identifier': MyCustomFeatureClassName}

    3. create a dict for this feature in getInstrumentFeatureConfigDicts() above. Dict format is:
            customFeatureDict = {'featureKey': 'my_custom_feature_key',
                                'featureId': 'my_custom_feature_identifier',
                                'params': {'param1': 'value1'}}
    You can now use this feature by calling it's featureKey, 'my_custom_feature_key' in getPrediction()
    '''
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        # Custom parameter which can be used as input to computation of this feature
        param1Value = featureParams['param1']

        # A holder for the all the instrument features
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()

        # dataframe for a historical instrument feature (basis in this case). The index is the timestamps
        # atmost upto lookback data points. The columns of this dataframe are the stocks/instrumentIds.
        lookbackInstrumentValue = lookbackInstrumentFeatures.getFeatureDf('adjClose')

        # The last row of the previous dataframe gives the last calculated value for that feature (basis in this case)
        # This returns a series with stocks/instrumentIds as the index.
        currentValue = lookbackInstrumentValue.iloc[-1]

        if param1Value == 'value1':
            return currentValue * 0.1
        else:
            return currentValue * 0.5

In [157]:
tf = MyFunctions()
tsParams = MyTradingParams(tf)
tradingSystem = TradingSystem(tsParams)
results = tradingSystem.startTrading()

results

Processing data for stock: SPY


KeyError: "name='B', domain=None, path=None"